In [1]:
import sys
sys.path.append('/home/daniel/Documents/Uni/MT/poi-prediction/')
import pandas as pd
from src.eval import create_self_training_pois

In [9]:
new_missing_fold_0 = create_self_training_pois(
    data_module_save_path= '/home/daniel/Documents/Uni/MT/experiment_logs/gruber/surface/all_pois/freeze/fold_0/SA-DenseNet-PatchTransformer/version_0/data_module_params.json',
    checkpoint_path='/home/daniel/Documents/Uni/MT/experiment_logs/gruber/surface/all_pois/freeze/fold_0/SA-DenseNet-PatchTransformer/version_0/checkpoints/sad-pt-epoch=30-fine_mean_distance_val=2.16.ckpt',
    poi_file_ending='pred_gen_0.json',
)
new_missing_fold_1 = create_self_training_pois(
    data_module_save_path= '/home/daniel/Documents/Uni/MT/experiment_logs/gruber/surface/all_pois/freeze/fold_1/SA-DenseNet-PatchTransformer/version_0/data_module_params.json',
    checkpoint_path='/home/daniel/Documents/Uni/MT/experiment_logs/gruber/surface/all_pois/freeze/fold_1/SA-DenseNet-PatchTransformer/version_0/checkpoints/sad-pt-epoch=44-fine_mean_distance_val=2.29.ckpt',
    poi_file_ending='pred_gen_0.json',
)
new_missing_fold_2 = create_self_training_pois(
    data_module_save_path='/home/daniel/Documents/Uni/MT/experiment_logs/gruber/surface/all_pois/freeze/fold_2/SA-DenseNet-PatchTransformer/version_0/data_module_params.json',
    checkpoint_path='/home/daniel/Documents/Uni/MT/experiment_logs/gruber/surface/all_pois/freeze/fold_2/SA-DenseNet-PatchTransformer/version_0/checkpoints/sad-pt-epoch=46-fine_mean_distance_val=2.39.ckpt',
    poi_file_ending='pred_gen_0.json',
)
new_missing_fold_3 = create_self_training_pois(
    data_module_save_path='/home/daniel/Documents/Uni/MT/experiment_logs/gruber/surface/all_pois/freeze/fold_3/SA-DenseNet-PatchTransformer/version_0/data_module_params.json',
    checkpoint_path='/home/daniel/Documents/Uni/MT/experiment_logs/gruber/surface/all_pois/freeze/fold_3/SA-DenseNet-PatchTransformer/version_0/checkpoints/sad-pt-epoch=31-fine_mean_distance_val=2.21.ckpt',
    poi_file_ending='pred_gen_0.json',
)
new_missing_fold_4 = create_self_training_pois(
    data_module_save_path='/home/daniel/Documents/Uni/MT/experiment_logs/gruber/surface/all_pois/freeze/fold_4/SA-DenseNet-PatchTransformer/version_0/data_module_params.json',
    checkpoint_path='/home/daniel/Documents/Uni/MT/experiment_logs/gruber/surface/all_pois/freeze/fold_4/SA-DenseNet-PatchTransformer/version_0/checkpoints/sad-pt-epoch=22-fine_mean_distance_val=2.39.ckpt',
    poi_file_ending='pred_gen_0.json',
)


In [11]:
old_master = pd.read_csv('/home/daniel/Data/Gruber/cutouts_scale-1-1-1/master_df_cleaned.csv')

old_master.head()

,subject,vertebra,file_dir,use_sample,bad_poi_list,n_bad_pois
0,WS-06,1,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,False,"[101, 102, 103, 104, 134, 136, 109, 111, 112, ...",14
1,WS-06,2,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,True,[],0
2,WS-06,3,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,True,[],0
3,WS-06,4,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,True,[],0
4,WS-06,5,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,True,[],0


In [13]:
missing_poi_df_0 = pd.DataFrame(new_missing_fold_0)
missing_poi_df_1 = pd.DataFrame(new_missing_fold_1)
missing_poi_df_2 = pd.DataFrame(new_missing_fold_2)
missing_poi_df_3 = pd.DataFrame(new_missing_fold_3)
missing_poi_df_4 = pd.DataFrame(new_missing_fold_4)

missing_poi_df = pd.concat([missing_poi_df_0, missing_poi_df_1, missing_poi_df_2, missing_poi_df_3, missing_poi_df_4])

In [14]:
missing_poi_df

,subject,vertebra,bad_poi_list
0,WS-18,3,"[109, 110, 127]"
1,WS-18,4,"[101, 110, 127]"
2,WS-18,5,"[101, 109, 110, 127, 149, 151]"
3,WS-18,6,"[109, 110, 117, 118, 141, 143, 149, 151]"
4,WS-18,7,"[109, 110, 127, 143, 144, 149, 151]"
...,...,...,...
121,WS-50,20,"[102, 117, 119, 127, 143]"
122,WS-50,21,"[117, 119]"
123,WS-50,22,"[109, 117, 119, 143]"
124,WS-50,23,"[117, 119, 120, 134, 136, 143]"


In [16]:
#Remove index from old_master
old_master.reset_index(drop=True, inplace=True)

In [18]:
#Join old master with missing_poi_list on subject and vertebra

new_master = pd.merge(old_master, missing_poi_df, on=['subject', 'vertebra'], how='left')

In [24]:
#Drop old bad_poi_list and rename new bad_poi_list to bad_poi_list
#new_master.drop(columns=['bad_poi_list_x'], inplace=True)
new_master.rename(columns={'bad_poi_list_y': 'bad_poi_list'}, inplace=True)

In [25]:
#Replace NaN values in bad_poi_list with empty list
new_master['bad_poi_list'].fillna(value='[]', inplace=True)

In [50]:
#Fill in missing commas for spaces in bad_poi_list
new_master['bad_poi_list'] = new_master['bad_poi_list'].apply(lambda x: x.replace(' ', ', '))

In [53]:
#Remove commas directly after brackets
new_master['bad_poi_list'] = new_master['bad_poi_list'].apply(lambda x: x.replace('[,', '['))

In [51]:
new_master.tail(50)

,subject,vertebra,file_dir,use_sample,n_bad_pois,bad_poi_list
808,WS-29,23,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,True,0,"[119, 127, 141, 143, 149, 151]"
809,WS-29,24,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,True,0,"[119, 141, 143, 149, 151]"
810,WS-00,1,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,True,13,[]
811,WS-00,2,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,True,2,[]
812,WS-00,3,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,True,2,"[119, 127, 143, 151]"
813,WS-00,4,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,True,0,"[125, 127, 143, 151]"
814,WS-00,5,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,True,1,"[109, 110, 111, 119, 141, 142, 143, 149, 151]"
815,WS-00,6,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,True,1,"[111, 127, 134, 141, 142, 143, 144, 149, 151]"
816,WS-00,7,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,True,1,"[119, 142, 151]"
817,WS-00,8,/home/daniel/Data/Gruber/cutouts_scale-1-1-1/W...,True,2,"[118, 119, 142, 144, 151]"


In [54]:
#Save new master to csv
new_master.to_csv('/home/daniel/Data/Gruber/cutouts_scale-1-1-1/master_df_pred_gen_0_cleaned.csv', index=False)